In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling
import re, os
from collections import OrderedDict

from oauth2client.client import GoogleCredentials
from google.colab import drive
gdrive_path = '/content/gdrive'
drive.mount(gdrive_path)

initial_path = '/content/gdrive/My Drive/Colab Notebooks/results/parameters/formatted_qbso_simple/'
datasets = ['Breastcancer','Glass','Hepatitis','Ionosphere','Movementlibras','Sonar','Spect','WDBC','Wine','Zoo']
data_size = [10,9,19,34,90,60,22,30,13,16]
test_params = ['bso/bees_number/','bso/flip/','bso/max_chance/','rl/alpha/','rl/epsilon/','rl/gamma/']

for branch in test_params:
  param = branch.split('/')[1]
  param_values = []
  index = 0
  for dataset in datasets:
    dataset_paths_for_param = set()
    for root, dirs, files in os.walk(initial_path + branch):
      if root.endswith(str(param + '/')) :
        param_values = dirs.copy()
      for name in files:
        full_file_path = os.path.join(root, name)
        if re.search(param,full_file_path) and re.search(dataset,full_file_path) and name.endswith('.xlsx'):
          for val in param_values:
            if re.search(val,full_file_path):
              dataset_paths_for_param.add(full_file_path)
    file = pd.DataFrame()
    tmp_data = pd.DataFrame()
    data = pd.DataFrame()
    vals_list = []
    for dataset_path in dataset_paths_for_param:
      file = pd.read_excel(dataset_path,index_col=None,skiprows=range(11,50))
      tmp_data = file[['Accuracy','N_Features','Time','Size_sol_space','Top_10%_features']]
      data = data.append(tmp_data, ignore_index=True)
      val = re.search('[0-9]+\.[0-9]+|[0-9]+', dataset_path)[0]
      vals_list += [val]*10
    data[param] = pd.Series(vals_list)
    mylist = []
    for item in data['Top_10%_features']:
        mlist = item.strip('[]').split(',')
        for nb in mlist:
          mylist.append(int(nb))
    plt.hist(mylist,bins=data_size[index])
    plt.ylabel('Number')
    plt.xlabel('Features')
    plt.title('Top 10% used features')
    #plt.savefig(initial_path + branch + 'Reporting/' + dataset + '.png')
    plt.close()
    index += 1

    #data.to_excel(initial_path + branch + 'Reporting/' + dataset + '.xlsx', index=None, header= True)
    #profile = pandas_profiling.ProfileReport(data)
    #profile.to_file(outputfile= initial_path + branch + 'Reporting/' + dataset + ".html")

In [1]:
!nvidia-smi

Wed Jun 12 16:46:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  